<a href="https://colab.research.google.com/github/YeimeLeandro/Hyperblog/blob/main/Punto_3_Examen_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import hilbert, svd

def generar_matriz_hilbert(n):
    """
    Genera una matriz de Hilbert de tamaño n x n utilizando scipy.linalg.hilbert.

    Parámetros:
    n (int): Tamaño de la matriz (n x n).

    Retorna:
    numpy.ndarray: Matriz de Hilbert de tamaño n x n.
    """
    return hilbert(n)

def norma_inducida_p_optimizacion(A, p, tol=1e-6, max_iter=1000):
    """
    Calcula la norma inducida de orden p de la matriz A utilizando optimización.

    Parámetros:
    A (numpy.ndarray): Matriz de entrada.
    p (float): Orden de la norma.
    tol (float): Tolerancia para la convergencia.
    max_iter (int): Número máximo de iteraciones.

    Retorna:
    float: Norma inducida de orden p.
    """
    n = A.shape[1]

    # Función objetivo: -||Ax||_p (negativa para maximizar)
    def func(x):
        Ax = A @ x
        return -np.linalg.norm(Ax, ord=p)

    # Restricción: ||x||_p = 1
    cons = {'type': 'eq', 'fun': lambda x: np.linalg.norm(x, ord=p) - 1}

    # Búsqueda inicial: vector aleatorio normalizado
    x0 = np.random.rand(n)
    x0 = x0 / np.linalg.norm(x0, ord=p)

    # Optimización
    res = minimize(func, x0, method='SLSQP', constraints=cons, options={'ftol': tol, 'maxiter': max_iter})

    # Verificar que la optimización fue exitosa
    if not res.success:
        raise ValueError(f"La optimización no convergió: {res.message}")

    norma_p = -res.fun
    return norma_p

def aproximar_norma_inducida_p_metodo_potencia(A, p, tol=1e-6, max_iter=1000):
    """
    Aproxima la norma inducida de orden p de una matriz A utilizando el método de la potencia generalizado.

    Parámetros:
    A (numpy.ndarray): Matriz de entrada.
    p (float): Orden de la norma.
    tol (float): Tolerancia para la convergencia.
    max_iter (int): Número máximo de iteraciones.

    Retorna:
    float: Norma inducida de orden p.
    numpy.ndarray: Vector unitario asociado.
    """
    n = A.shape[1]
    x = np.random.rand(n)
    x = x / np.linalg.norm(x, ord=p)
    norma_anterior = 0

    for k in range(max_iter):
        y = A @ x
        norma_p = np.linalg.norm(y, ord=p)
        if norma_p == 0:
            raise ValueError("La norma de y es cero. Se detiene la iteración.")

        x_new = y / norma_p

        if abs(norma_p - norma_anterior) < tol:
            print(f"Convergencia alcanzada en {k+1} iteraciones.")
            break

        x = x_new
        norma_anterior = norma_p
    else:
        print("Se alcanzó el número máximo de iteraciones sin convergencia.")

    return norma_p, x_new

def calcular_normas_inducidas_hilbert(n, p_values):
    """
    Calcula las normas inducidas de una matriz de Hilbert para diferentes valores de p.

    Parámetros:
    n (int): Tamaño de la matriz de Hilbert (n x n).
    p_values (list): Lista de valores de p para los cuales calcular las normas.

    Retorna:
    dict: Diccionario con las normas inducidas calculadas para cada p.
    """
    # Generar la matriz de Hilbert
    H = generar_matriz_hilbert(n)

    # Verificación de la matriz generada
    print("Matriz de Hilbert H10 (10x10):\n")
    print(H)

    # Diccionario para almacenar las normas
    normas = {}

    # Calcular la norma inducida para cada valor de p
    for p in p_values:
        if p == np.inf:
            p_display = '∞'
        else:
            p_display = str(p)

        print(f"\nCalculando la norma inducida para p = {p_display}...")

        if p == 1:
            # Norma inducida de orden 1: máximo de las sumas de las columnas
            norma = np.max(np.sum(np.abs(H), axis=0))
            print(f"Norma inducida exacta para p = {p_display}: {norma:.6f}")
        elif p == 2:
            # Norma inducida de orden 2: norma espectral utilizando SVD
            singular_values = svd(H, compute_uv=False)
            norma = singular_values[0]
            print(f"Norma inducida exacta para p = {p_display}: {norma:.6f}")
            print("\nValores singulares de H10:")
            print(singular_values)
        elif p == np.inf:
            # Norma inducida de orden ∞: máximo de las sumas de las filas
            norma = np.max(np.sum(np.abs(H), axis=1))
            print(f"Norma inducida exacta para p = {p_display}: {norma:.6f}")
        else:
            # Norma inducida para otros valores de p utilizando optimización
            try:
                norma = norma_inducida_p_optimizacion(H, p)
                print(f"Norma inducida aproximada para p = {p_display}: {norma:.6f}")
            except ValueError as e:
                print(e)
                continue

        normas[p_display] = norma

    return normas

def main():
    # Valores de p a evaluar
    p_values = [1, 2, 3, np.inf]

    # Tamaño de la matriz de Hilbert
    n = 10

    # Calcular y mostrar las normas inducidas
    normas_hilbert = calcular_normas_inducidas_hilbert(n, p_values)

    # Resumen de los resultados
    print("\nResumen de las normas inducidas calculadas:")
    for p, norma in normas_hilbert.items():
        print(f"Norma inducida para p = {p}: {norma:.6f}")

    # Aproximación de la norma inducida de orden 3 utilizando el método de la potencia
    print("\nAproximando la norma inducida para p = 3 utilizando el método de la potencia...")
    H = generar_matriz_hilbert(n)
    norma_3_potencia, vector_3_potencia = aproximar_norma_inducida_p_metodo_potencia(H, p=3)
    print(f"Norma inducida aproximada para p = 3 (método de la potencia): {norma_3_potencia:.6f}")
    print(f"Vector unitario asociado para norma 3:\n{vector_3_potencia}")

    # Verificación de la norma del vector asociado para p=3
    norma_vector_3 = np.linalg.norm(vector_3_potencia, ord=3)
    print(f"\nNorma del vector asociado para norma 3: {norma_vector_3:.6f}")

if __name__ == "__main__":
    main()


Matriz de Hilbert H10 (10x10):

[[1.         0.5        0.33333333 0.25       0.2        0.16666667
  0.14285714 0.125      0.11111111 0.1       ]
 [0.5        0.33333333 0.25       0.2        0.16666667 0.14285714
  0.125      0.11111111 0.1        0.09090909]
 [0.33333333 0.25       0.2        0.16666667 0.14285714 0.125
  0.11111111 0.1        0.09090909 0.08333333]
 [0.25       0.2        0.16666667 0.14285714 0.125      0.11111111
  0.1        0.09090909 0.08333333 0.07692308]
 [0.2        0.16666667 0.14285714 0.125      0.11111111 0.1
  0.09090909 0.08333333 0.07692308 0.07142857]
 [0.16666667 0.14285714 0.125      0.11111111 0.1        0.09090909
  0.08333333 0.07692308 0.07142857 0.06666667]
 [0.14285714 0.125      0.11111111 0.1        0.09090909 0.08333333
  0.07692308 0.07142857 0.06666667 0.0625    ]
 [0.125      0.11111111 0.1        0.09090909 0.08333333 0.07692308
  0.07142857 0.06666667 0.0625     0.05882353]
 [0.11111111 0.1        0.09090909 0.08333333 0.07692308 0.0